## Resolve index


In [1]:
import polars as pl
from pathlib import Path
from pycomfort.files import tprint

base = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
data_dir = base / "data"
input_dir = data_dir / "input"
output_dir = data_dir / "output"
tprint(output_dir)

output
	atomica_index.parquet
	PP.csv
	PRNA.jsonl.gz
	.~lock.PP.csv#
	Ppeptide.csv
	PP.jsonl.gz
	atomica_index_old.parquet
	PP_embeddings.parquet
	Ppeptide.jsonl.gz
	PP_extended.parquet
	PRNA.csv
	PP_advanced.parquet
	.gitignore
	idmapping_selected.parquet
	protein_names.parquet
	PP_graph_embeddings.parquet
	pdbs
		README.md
		1TGR.pdb
		4XSS.pdb


In [2]:
# Configure Polars to show all rows and strings without truncation
pl.Config.set_tbl_rows(-1)  # Show all rows
pl.Config.set_fmt_str_lengths(1000000)  # Don't truncate strings
pl.Config.set_tbl_cols(-1)  # Show all columns
pl.Config.set_tbl_width_chars(1000)  # Set wide table width


polars.config.Config

In [3]:
from pathlib import Path

base = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
data_dir = base / "data"
input_dir = data_dir / "input"
output_dir = data_dir / "output"
tprint(output_dir)

output
	atomica_index.parquet
	PP.csv
	PRNA.jsonl.gz
	.~lock.PP.csv#
	Ppeptide.csv
	PP.jsonl.gz
	atomica_index_old.parquet
	PP_embeddings.parquet
	Ppeptide.jsonl.gz
	PP_extended.parquet
	PRNA.csv
	PP_advanced.parquet
	.gitignore
	idmapping_selected.parquet
	protein_names.parquet
	PP_graph_embeddings.parquet
	pdbs
		README.md
		1TGR.pdb
		4XSS.pdb


In [4]:
import polars as pl
from pathlib import Path

df = pl.read_parquet(output_dir / "atomica_index.parquet")
print(f"Total rows: {len(df)}")
print(f"Columns: {df.columns}")
df.head()


Total rows: 83
Columns: ['pdb_id', 'cif_path', 'metadata_path', 'summary_path', 'critical_residues_path', 'interact_scores_path', 'pymol_path', 'critical_residues_count', 'total_time_seconds', 'gpu_memory_mb_max', 'metadata_found', 'title', 'uniprot_ids', 'organisms', 'taxonomy_ids', 'gene_symbols', 'structures_json']


pdb_id,cif_path,metadata_path,summary_path,critical_residues_path,interact_scores_path,pymol_path,critical_residues_count,total_time_seconds,gpu_memory_mb_max,metadata_found,title,uniprot_ids,organisms,taxonomy_ids,gene_symbols,structures_json
str,str,str,str,str,str,str,i64,f64,f64,bool,str,list[str],list[null],list[null],list[str],str
"""1B68""","""1b68/1b68.cif""","""1b68/1b68_metadata.json""","""1b68/1b68_summary.json""","""1b68/1b68_critical_residues.tsv""","""1b68/1b68_interact_scores.json""","""1b68/1b68_pymol_commands.pml""",138,22.0,1035.62,true,"""APOLIPOPROTEIN E4 (APOE4), 22K FRAGMENT""","[""P02649""]",[],[],"[""APOE""]","""[{""structure_id"": ""1B68"", ""uniprot_id"": ""P02649"", ""gene_symbol"": ""APOE"", ""deposition_date"": ""1999-01-21"", ""experimental_method"": ""X-RAY DIFFRACTION"", ""resolution"": 2, ""r_free"": null, ""pdb_redo_available"": false, ""pdb_redo_rfree"": null, ""chains"": [], ""coverage"": {}, ""warnings"": [], ""complex_info"": {""has_protein_complex"": false, ""protein_complex_details"": null, ""has_nucleotide"": false, ""nucleotide_details"": null, ""has_ligand"": false, ""ligand_details"": null, ""is_fusion"": false}}]"""
"""1LE2""","""1le2/1le2.cif""","""1le2/1le2_metadata.json""","""1le2/1le2_summary.json""","""1le2/1le2_critical_residues.tsv""","""1le2/1le2_interact_scores.json""","""1le2/1le2_pymol_commands.pml""",144,22.99,1060.61,true,"""STRUCTURAL BASIS FOR ALTERED FUNCTION IN THE COMMON MUTANTS OF HUMAN APOLIPOPROTEIN-E""","[""P02649""]",[],[],"[""APOE""]","""[{""structure_id"": ""1LE2"", ""uniprot_id"": ""P02649"", ""gene_symbol"": ""APOE"", ""deposition_date"": ""1991-08-22"", ""experimental_method"": ""X-RAY DIFFRACTION"", ""resolution"": 3, ""r_free"": null, ""pdb_redo_available"": false, ""pdb_redo_rfree"": null, ""chains"": [], ""coverage"": {}, ""warnings"": [], ""complex_info"": {""has_protein_complex"": false, ""protein_complex_details"": null, ""has_nucleotide"": false, ""nucleotide_details"": null, ""has_ligand"": false, ""ligand_details"": null, ""is_fusion"": false}}]"""
"""1LE4""","""1le4/1le4.cif""","""1le4/1le4_metadata.json""","""1le4/1le4_summary.json""","""1le4/1le4_critical_residues.tsv""","""1le4/1le4_interact_scores.json""","""1le4/1le4_pymol_commands.pml""",139,22.18,1045.54,true,"""STRUCTURAL BASIS FOR ALTERED FUNCTION IN THE COMMON MUTANTS OF HUMAN APOLIPOPROTEIN-E""","[""P02649""]",[],[],"[""APOE""]","""[{""structure_id"": ""1LE4"", ""uniprot_id"": ""P02649"", ""gene_symbol"": ""APOE"", ""deposition_date"": ""1991-08-22"", ""experimental_method"": ""X-RAY DIFFRACTION"", ""resolution"": 2.5, ""r_free"": null, ""pdb_redo_available"": false, ""pdb_redo_rfree"": null, ""chains"": [], ""coverage"": {}, ""warnings"": [], ""complex_info"": {""has_protein_complex"": false, ""protein_complex_details"": null, ""has_nucleotide"": false, ""nucleotide_details"": null, ""has_ligand"": false, ""ligand_details"": null, ""is_fusion"": false}}]"""
"""1LPE""","""1lpe/1lpe.cif""","""1lpe/1lpe_metadata.json""","""1lpe/1lpe_summary.json""","""1lpe/1lpe_critical_residues.tsv""","""1lpe/1lpe_interact_scores.json""","""1lpe/1lpe_pymol_commands.pml""",144,23.11,1063.86,true,"""THREE-DIMENSIONAL STRUCTURE OF THE LDL RECEPTOR-BINDING DOMAIN OF HUMAN APOLIPOPROTEIN E""","[""P02649""]",[],[],"[""APOE""]","""[{""structure_id"": ""1LPE"", ""uniprot_id"": ""P02649"", ""gene_symbol"": ""APOE"", ""deposition_date"": ""1991-08-22"", ""experimental_method"": ""X-RAY DIFFRACTION"", ""resolution"": 2.25, ""r_free"": null, ""pdb_redo_available"": false, ""pdb_redo_rfree"": null, ""chains"": [], ""coverage"": {}, ""warnings"": [], ""complex_info"": {""has_protein_complex"": false, ""protein_complex_details"": null, ""has_nucleotide"": false, ""nucleotide_details"": null, ""has_ligand"": false, ""ligand_details"": null, ""is_fusion"": false}}]"""
"""1NFN""","""1nfn/1nfn.cif""","""1nfn/1nfn_metadata.json""","""1nfn/1nfn_summary.json""","""1nfn/1nfn_critical_residues.tsv""","""1nfn/1nfn_interact_sc

In [ ]:
# Check for proteins with multiple species
proteins_with_multiple_species = (
    df.group_by("protein_name")
    .agg([
        pl.col("species").n_unique().alias("species_count"),
        pl.col("species").unique().alias("species_list")
    ])
    .filter(pl.col("species_count") > 1)
    .sort("species_count", descending=True)
)

print(f"\nProteins with multiple species: {len(proteins_with_multiple_species)}")
proteins_with_multiple_species


In [ ]:
# Check for proteins with multiple UniProt IDs
proteins_with_multiple_uniprots = (
    df.group_by("protein_name")
    .agg([
        pl.col("uniprot_id").n_unique().alias("uniprot_count"),
        pl.col("uniprot_id").unique().alias("uniprot_list")
    ])
    .filter(pl.col("uniprot_count") > 1)
    .sort("uniprot_count", descending=True)
)

print(f"\nProteins with multiple UniProt IDs: {len(proteins_with_multiple_uniprots)}")
proteins_with_multiple_uniprots


In [ ]:
# Check for proteins with both multiple species AND multiple UniProt IDs
proteins_with_both = (
    df.group_by("protein_name")
    .agg([
        pl.col("species").n_unique().alias("species_count"),
        pl.col("uniprot_id").n_unique().alias("uniprot_count"),
        pl.col("species").unique().alias("species_list"),
        pl.col("uniprot_id").unique().alias("uniprot_list")
    ])
    .filter((pl.col("species_count") > 1) & (pl.col("uniprot_count") > 1))
    .sort(["species_count", "uniprot_count"], descending=True)
)

print(f"\nProteins with BOTH multiple species AND multiple UniProt IDs: {len(proteins_with_both)}")
proteins_with_both


In [ ]:
# Get detailed view of a sample protein with multiple entries
if len(proteins_with_both) > 0:
    sample_protein = proteins_with_both[0, "protein_name"]
    print(f"\nDetailed view of protein: {sample_protein}")
    df.filter(pl.col("protein_name") == sample_protein).select([
        "protein_name", "uniprot_id", "species", "pdb_id", "organism"
    ])
elif len(proteins_with_multiple_species) > 0:
    sample_protein = proteins_with_multiple_species[0, "protein_name"]
    print(f"\nDetailed view of protein with multiple species: {sample_protein}")
    df.filter(pl.col("protein_name") == sample_protein).select([
        "protein_name", "uniprot_id", "species", "pdb_id", "organism"
    ])
elif len(proteins_with_multiple_uniprots) > 0:
    sample_protein = proteins_with_multiple_uniprots[0, "protein_name"]
    print(f"\nDetailed view of protein with multiple UniProt IDs: {sample_protein}")
    df.filter(pl.col("protein_name") == sample_protein).select([
        "protein_name", "uniprot_id", "species", "pdb_id", "organism"
    ])
else:
    print("\nNo proteins with multiple species or UniProt IDs found")


In [ ]:
# Summary statistics
print("\n=== SUMMARY ===")
print(f"Total unique protein names: {df['protein_name'].n_unique()}")
print(f"Total unique UniProt IDs: {df['uniprot_id'].n_unique()}")
print(f"Total unique species: {df['species'].n_unique()}")
print(f"Total PDB entries: {len(df)}")
print(f"\nProteins with multiple species: {len(proteins_with_multiple_species)}")
print(f"Proteins with multiple UniProt IDs: {len(proteins_with_multiple_uniprots)}")
print(f"Proteins with both: {len(proteins_with_both)}")
